# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,67.59,76,91,16.87,PN,1690197234
1,1,puerto ayora,-0.7393,-90.3518,76.10,85,51,7.74,EC,1690197274
2,2,vorgashor,67.5833,63.9500,71.91,50,46,5.53,RU,1690197274
3,3,kodiak,57.7900,-152.4072,47.71,93,0,4.61,US,1690197274
4,4,port-aux-francais,-49.3500,70.2167,41.72,91,100,32.08,TF,1690197234


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
   "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size = "Humidity",
    color = "City",
   alpha=0.5,
)
    


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 80) & (city_data_df["Max Temp"] >= 70)]

#   * Wind speed less than 10 mph.
city_data_df = city_data_df.loc[city_data_df["Wind Speed"] < 10]

#   * Zero cloudiness.
city_data_df = city_data_df.loc[city_data_df["Cloudiness"] == 0]


# Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,bethel,41.3712,-73.4140,70.92,85,0,0.00,US,1690197275
21,21,snyder,32.7179,-100.9176,79.32,43,0,4.61,US,1690197276
47,47,remire-montjoly,4.9167,-52.2667,77.04,94,0,0.00,GF,1690197280
49,49,great falls,47.5002,-111.3008,73.42,46,0,6.91,US,1690197043
51,51,matoury,4.8483,-52.3317,76.96,94,0,0.00,GF,1690197280


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity.
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,bethel,US,41.3712,-73.4140,85,
21,snyder,US,32.7179,-100.9176,43,
47,remire-montjoly,GF,4.9167,-52.2667,94,
49,great falls,US,47.5002,-111.3008,46,
51,matoury,GF,4.8483,-52.3317,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 5000
params = {
    #"radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()

    print(name_address)
    

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hampton Inn Danbury', 'country': 'United States', 'country_code': 'us', 'state': 'Connecticut', 'county': 'Western Connecticut Planning Region', 'city': 'Danbury', 'hamlet': 'Beaverbrook', 'postcode': '06801', 'street': 'Industrial Plaza Road', 'lon': -73.41135845863734, 'lat': 41.409656749999996, 'state_code': 'CT', 'formatted': 'Hampton Inn Danbury, Industrial Plaza Road, Danbury, CT 06801, United States of America', 'address_line1': 'Hampton Inn Danbury', 'address_line2': 'Industrial Plaza Road, Danbury, CT 06801, United States of America', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Hampton Inn Danbury', 'bran

,City,Country,Lat,Lng,Humidity,Hotel Name
13,bethel,US,41.3712,-73.4140,85,Hampton Inn Danbury
21,snyder,US,32.7179,-100.9176,43,Days Inn
47,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
49,great falls,US,47.5002,-111.3008,46,Hotel Arvon
51,matoury,GF,4.8483,-52.3317,94,La Marmotte
76,mapimi,MX,25.8167,-103.8500,59,No hotel found
85,altamira,BR,-3.2033,-52.2064,100,Hotel Vale do Xingu
134,bryan,US,30.6744,-96.3700,88,The Lasalle Hotel
151,palmer,US,42.1584,-72.3287,87,No hotel found
224,tolanaro,MG,-25.0319,46.9987,65,Hôtel Mahavokey


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_1= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size = "Humidity",
    color = "City",
    alpha=0.5,
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)